In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout

# Set the path to your dataset
train_dir = '/content/drive/MyDrive/emerg vs non emer'
test_dir = '/content/drive/MyDrive/test'

# Define the CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    #tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    #tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    #tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid'),


])

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Preprocess and augment the training data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(64, 64),
                                                    batch_size=20,
                                                    class_mode='binary')

# Preprocess the test data (only rescaling)
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(64, 64),
                                                  batch_size=20,
                                                  class_mode='binary')




Found 1935 images belonging to 2 classes.
Found 342 images belonging to 2 classes.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model.fit(train_generator,steps_per_epoch=77,epochs=60)

# Save the trained model
model.save('emergency_classifier.h5')

Epoch 1/60
77/77 [==============================] - 250s 3s/step - loss: 0.6054 - accuracy: 0.7075
Epoch 2/60
77/77 [==============================] - 54s 686ms/step - loss: 0.6098 - accuracy: 0.7068
Epoch 3/60
77/77 [==============================] - 20s 264ms/step - loss: 0.5965 - accuracy: 0.7094
Epoch 4/60
77/77 [==============================] - 8s 103ms/step - loss: 0.5578 - accuracy: 0.7325
Epoch 5/60
77/77 [==============================] - 9s 113ms/step - loss: 0.5336 - accuracy: 0.7694
Epoch 6/60
77/77 [==============================] - 8s 98ms/step - loss: 0.5165 - accuracy: 0.7805
Epoch 7/60
77/77 [==============================] - 6s 83ms/step - loss: 0.4921 - accuracy: 0.7876
Epoch 8/60
77/77 [==============================] - 6s 83ms/step - loss: 0.4754 - accuracy: 0.7902
Epoch 9/60
77/77 [==============================] - 7s 95ms/step - loss: 0.4535 - accuracy: 0.8059
Epoch 10/60
77/77 [==============================] - 6s 84ms/step - loss: 0.4581 - accuracy: 0.8091
Epo

In [ ]:
import numpy as np

# Load the saved model
model = tf.keras.models.load_model('emergency_classifier.h5')

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator)

# Predict the classes for the test data
predictions = model.predict(test_generator)
predicted_classes = np.round(predictions).flatten()

# Calculate the precision metric
true_classes = test_generator.classes
precision = np.sum(predicted_classes == true_classes) / len(true_classes)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)
print('Precision:', precision)


18/18 [==============================] - 1s 52ms/step
Test Loss: 0.24744488298892975
Test Accuracy: 0.9122806787490845
Precision: 0.5964912280701754


In [ ]:
from sklearn.metrics import recall_score
recall = recall_score(true_classes, predicted_classes)
print('Recall:', recall)


Recall: 0.746606334841629


In [ ]:
import numpy as np
from keras.preprocessing import image
import keras.utils as image
test_image=image.load_img('/content/drive/MyDrive/emerg vs non emer/emergency/426.jpg',target_size=(64,64))
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
result=model.predict(test_image)
train_generator.class_indices
if result[0][0]==1:
  prediction='Non emergency vehicle'
else:
  prediction='Emergency Vehicle'
print(prediction)

1/1 [==============================] - 0s 21ms/step
Emergency Vehicle


# New Section

# New Section

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.models import Model

# Set the path to your dataset
train_dir = '/content/drive/MyDrive/emerg vs non emer'
test_dir = '/content/drive/MyDrive/test'

# Load the ResNet50 model
resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers in the ResNet50 model
for layer in resnet50.layers:
    layer.trainable = False

# Create a new model based on ResNet50
model = tf.keras.models.Sequential([
    resnet50,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Preprocess and augment the training data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(224, 224),
                                                    batch_size=20,
                                                    class_mode='binary')

# Preprocess the test data (only rescaling)
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(224, 224),
                                                  batch_size=20,
                                                  class_mode='binary')

# Train the model
model.fit(train_generator, steps_per_epoch=77, epochs=60)

# Save the trained model
model.save('emergency_classifier_resnet50.h5')

In [ ]:
import pickle
pickle.dump(model,open('/content/emergency_classifier_resnet50.h5','wb'))

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.models import Model

# Set the path to your dataset
train_dir = '/content/drive/MyDrive/emerg vs non emer'
test_dir = '/content/drive/MyDrive/test'

# Load the MobileNetV2 model
mobilenetv2 = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers in the MobileNetV2 model
for layer in mobilenetv2.layers:
    layer.trainable = False

# Create a new model based on MobileNetV2
model = tf.keras.models.Sequential([
    mobilenetv2,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Preprocess and augment the training data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(224, 224),
                                                    batch_size=20,
                                                    class_mode='binary')

# Preprocess the test data (only rescaling)
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(224, 224),
                                                  batch_size=20,
                                                  class_mode='binary')

# Train the model
model.fit(train_generator, steps_per_epoch=77, epochs=60)

# Save the trained model
model.save('emergency_classifier_mobilenetv2.h5')

In [ ]:
import pickle
pickle.dump(model,open('emergency_classifier_mobilenetv2.h5','wb'))

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import NASNetMobile
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.models import Model

# Set the path to your dataset
train_dir = '/content/drive/MyDrive/emerg vs non emer'
test_dir = '/content/drive/MyDrive/test'

# Load the NASNetMobile model
nasnet = NASNetMobile(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers in the NASNetMobile model
for layer in nasnet.layers:
    layer.trainable = False

# Create a new model based on NASNetMobile
model = tf.keras.models.Sequential([
    nasnet,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Preprocess and augment the training data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(224, 224),
                                                    batch_size=20,
                                                    class_mode='binary')

# Preprocess the test data (only rescaling)
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(224, 224),
                                                  batch_size=20,
                                                  class_mode='binary')

# Train the model
model.fit(train_generator, steps_per_epoch=77, epochs=60)

# Save the trained model
model.save('emergency_classifier_nasnetmobile.h5')


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import SqueezeNet
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.models import Model

# Set the path to your dataset
train_dir = '/content/drive/MyDrive/emerg vs non emer'
test_dir = '/content/drive/MyDrive/test'
# Load the SqueezeNet model
squeezenet = SqueezeNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers in the SqueezeNet model
for layer in squeezenet.layers:
    layer.trainable = False

# Create a new model based on SqueezeNet
model = tf.keras.models.Sequential([
    squeezenet,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Preprocess and augment the training data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(224, 224),
                                                    batch_size=20,
                                                    class_mode='binary')

# Preprocess the test data (only rescaling)
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(224, 224),
                                                  batch_size=20,
                                                  class_mode='binary')

# Train the model
model.fit(train_generator, steps_per_epoch=77, epochs=60)

# Save the trained model
model.save('emergency_classifier_squeezenet.h5')
